In [96]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
import pandas as pd
import numpy as np
import psycopg2

### Métodos de la librería

In [98]:
#Creamos una variable para crear la conexión.
conn = psycopg2.connect(
    dbname = "online_sales",
    user = "my_user",
    password = "admin",
    host = "localhost",
    port = "5432"
)

In [99]:
cur = conn.cursor()

In [100]:
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 14.12 (Homebrew) on aarch64-apple-darwin23.4.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit',)

In [101]:
insert_query = """
                  INSERT INTO clientes(id_cliente, nombre, apellido, fecha_nacimiento, mail)
                  VALUES('123abc','Jaime','Rollon', '06-05-2000', 'ejemplo@gmail.com')
                
"""
cur.execute(insert_query)


In [102]:
query_elimination = """
DELETE FROM clientes WHERE id_cliente = %s
"""

cur.execute(query_elimination, ('123abc', ))


In [103]:
conn.rollback()

In [104]:
#Tabla clientes
df_ventas = pd.read_csv("../datos/dataset_ventas_limpio.csv")
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_venta                20000 non-null  object 
 1   fecha_venta             20000 non-null  object 
 2   fecha_entrega_prevista  20000 non-null  object 
 3   fecha_entrega_real      20000 non-null  object 
 4   id_cliente              20000 non-null  object 
 5   fecha_nacimiento        20000 non-null  object 
 6   mail                    20000 non-null  object 
 7   categoria_producto      20000 non-null  object 
 8   nombre_producto         20000 non-null  object 
 9   precio_unitario         20000 non-null  float64
 10  descuento               20000 non-null  float64
 11  cantidad_comprada       20000 non-null  int64  
 12  municipio               20000 non-null  object 
 13  monto_total             20000 non-null  float64
 14  nombre                  20000 non-null

In [105]:
tabla_clientes = df_ventas[['id_cliente', 'nombre', 'apellido', 'fecha_nacimiento', 'mail']]
tabla_clientes

,id_cliente,nombre,apellido,fecha_nacimiento,mail
0,77150ed6-c644-42d4-857e-fc73890b808c,Zoraida,Roselló-Piquer,1964-07-08,zoraidaroselló-piquer@mail.com
1,0728f6c5-6fc4-4370-82c1-fb25b1b8a2a7,Hermenegildo,Alegria Pi,1961-02-03,hermenegildoalegriapi@mail.com
2,23178f23-f716-49f2-9db3-e2808debf0e3,Matías,Solís Guardiola,1997-11-09,matíassolísguardiola@mail.com
3,65da15f0-bd12-4db0-ba7f-25a8cbd3375c,Cristóbal,Sastre Escudero,1976-11-21,cristóbalsastreescudero@mail.com
4,03681f50-b1d0-486b-b66b-804465c9a4b7,Hipólito,Sedano Canet,1973-09-09,hipólitosedanocanet@mail.com
...,...,...,...,...,...
19995,07b1c26f-c6a5-49ae-9e02-f8738a365bc5,Gema,Mateu Maldonado,1967-07-29,gemamateumaldonado@mail.com
19996,ab20d7bb-5c94-406a-9d1c-da101c4b8c15,Aureliano,de Neira,1973-09-03,aurelianodeneira@mail.com
19997,0f821aa7-4e46-45ff-b4f3-7e45a96aba5c,Manolo,Murcia Solsona,1970-05-20,manolomurciasolsona@mail.com
19998,632b83ef-d62a-4fae-b084-b29fdd10478a,Jenny,Cerdá-Olivares,1992-02-15,jennycerdá-olivares@mail.com


In [106]:
tabla_clientes = tabla_clientes.drop_duplicates()

In [107]:
tabla_clientes.sample(10)

,id_cliente,nombre,apellido,fecha_nacimiento,mail
7800,abe9d921-884a-4191-9301-0143e95bbfcb,Susanita,de Casares,1983-03-22,susanitadecasares@mail.com
493,9136d8d2-6383-4971-b5c7-371b9e9ee1d7,Telmo,Hidalgo Solera,1989-10-29,telmohidalgosolera@mail.com
1500,c19d8c73-5548-450a-8086-589c9c85ced3,Óscar,Rosselló Abril,1961-01-17,óscarrossellóabril@mail.com
1368,77a0ddce-8a28-43c9-81d1-c5a38e994758,Hortensia,Serra Guzman,1971-08-02,hortensiaserraguzman@mail.com
9919,557c524b-7bb5-4b71-80f9-fb33fd595d0d,Guiomar,Salazar Jara,1990-11-02,guiomarsalazarjara@mail.com
338,58e22293-f29e-4b69-b8f7-f4ab1133cac2,Flavio,del Gimeno,1980-04-25,flaviodelgimeno@mail.com
7051,a58f9c8d-592b-480d-a844-bee537ec4334,Florentino,Vera Isern,1998-12-01,florentinoveraisern@mail.com
262,d09c7cf0-728f-4f55-bce8-2312c5835d50,Pastora,Manzano Gibert,2000-01-02,pastoramanzanogibert@mail.com
1295,bb77c631-1f72-4d9a-9923-3ef10c4f5963,Martín,Aparicio Márquez,1963-12-18,martínapariciomárquez@mail.com
11459,d52c2a46-e7eb-446b-b953-7c9b7a69dcbb,Marta,Palacio Toro,1962-11-11,martapalaciotoro@mail.com


In [108]:
insert_query ="""
                  INSERT INTO clientes(id_cliente, nombre, apellido, fecha_nacimiento, mail)
                  VALUES(%s, %s, %s, %s, %s)               
"""


In [109]:
data_to_insert = [] 

for index, row in tabla_clientes.iterrows():
    data_to_insert.append((row['id_cliente'], row['nombre'], row['apellido'], row['fecha_nacimiento'], row['mail']))

# Verifica los primeros datos antes de insertarlos
print(data_to_insert[:3])

[('77150ed6-c644-42d4-857e-fc73890b808c', 'Zoraida', 'Roselló-Piquer', '1964-07-08', 'zoraidaroselló-piquer@mail.com'), ('0728f6c5-6fc4-4370-82c1-fb25b1b8a2a7', 'Hermenegildo', 'Alegria Pi', '1961-02-03', 'hermenegildoalegriapi@mail.com'), ('23178f23-f716-49f2-9db3-e2808debf0e3', 'Matías', 'Solís Guardiola', '1997-11-09', 'matíassolísguardiola@mail.com')]


In [110]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "clientes_pkey"
DETAIL:  Key (id_cliente)=(77150ed6-c644-42d4-857e-fc73890b808c) already exists.


In [ ]:
conn.rollback()

In [ ]:
#Query de comprobación:
query_com = """ 
                    SELECT * FROM clientes
"""
cur.execute(query_com)
cur.fetchmany(3)

[('77150ed6-c644-42d4-857e-fc73890b808c',
  'Zoraida',
  'Roselló-Piquer',
  datetime.date(1964, 7, 8),
  'zoraidaroselló-piquer@mail.com'),
 ('0728f6c5-6fc4-4370-82c1-fb25b1b8a2a7',
  'Hermenegildo',
  'Alegria Pi',
  datetime.date(1961, 2, 3),
  'hermenegildoalegriapi@mail.com'),
 ('23178f23-f716-49f2-9db3-e2808debf0e3',
  'Matías',
  'Solís Guardiola',
  datetime.date(1997, 11, 9),
  'matíassolísguardiola@mail.com')]

In [ ]:
#Cargamos el archivo
df_region = pd.read_csv("../datos/muni_provincia_ccaa.csv", )
df_region.head()

,municipio,provincia,comunidad_autonoma
0,Madrid,Comunidad de Madrid,Comunidad de Madrid
1,Murcia,Región de Murcia,Región de Murcia
2,Palma de Mallorca,Islas Baleares,Islas Baleares
3,Gijón,Principado de Asturias,Principado de Asturias
4,Oviedo,Principado de Asturias,Principado de Asturias


In [ ]:
tabla_ccaa = pd.DataFrame(df_region["comunidad_autonoma"].unique(),
                          columns=["comunidad_autonoma"])

tabla_ccaa

,comunidad_autonoma
0,Comunidad de Madrid
1,Región de Murcia
2,Islas Baleares
3,Principado de Asturias
4,Navarra
5,Almería
6,Cantabria
7,La Rioja
8,Melilla
9,Ceuta


In [ ]:
insert_query = """
                    INSERT INTO comunidades_autonomas (ccaa)
                    VALUES (%s)
                    """

In [ ]:
data_to_insert = [(row["comunidad_autonoma"],) for _, row in tabla_ccaa.iterrows()]


In [ ]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
query_comprobacion = """
                        SELECT * FROM comunidades_autonomas 
"""
cur.execute(query_comprobacion)
cur.fetchall()


[(1, 'Comunidad de Madrid'),
 (2, 'Región de Murcia'),
 (3, 'Islas Baleares'),
 (4, 'Principado de Asturias'),
 (5, 'Navarra'),
 (6, 'Almería'),
 (7, 'Cantabria'),
 (8, 'La Rioja'),
 (9, 'Melilla'),
 (10, 'Ceuta')]

In [ ]:
conn.rollback()

## Provincias

In [ ]:
df_region.columns

Index(['municipio', 'provincia', 'comunidad_autonoma'], dtype='object')

In [115]:
cur.execute("SELECT ccaa, id_ccaa FROM comunidades_autonomas")
ccaa_dict = dict(cur.fetchall())
ccaa_dict

{'Comunidad de Madrid': 1,
 'Región de Murcia': 2,
 'Islas Baleares': 3,
 'Principado de Asturias': 4,
 'Navarra': 5,
 'Almería': 6,
 'Cantabria': 7,
 'La Rioja': 8,
 'Melilla': 9,
 'Ceuta': 10}

In [114]:
conn.rollback()

In [ ]:
df_provincias = df_region[["provincia", "comunidad_autonoma"]].drop_duplicates()
df_provincias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 27
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   provincia           10 non-null     object
 1   comunidad_autonoma  10 non-null     object
dtypes: object(2)
memory usage: 240.0+ bytes


In [ ]:
data_to_insert = []

for _, row in df_provincias.iterrows():
    provincia = row["provincia"]
    comunidad_autonoma = row["comunidad_autonoma"]
    id_ccaa = ccaa_dict.get(comunidad_autonoma)
    data_to_insert.append([provincia, id_ccaa])
len(data_to_insert)



10

In [ ]:
insert_query = """ 
                    INSERT INTO provincias (provincia, id_ccaa)
                    values (%s, %s)
"""

cur.executemany(insert_query, data_to_insert)



In [ ]:
conn.commit()

## Municipios

In [123]:
df_region.head()

,municipio,provincia,comunidad_autonoma
0,Madrid,Comunidad de Madrid,Comunidad de Madrid
1,Murcia,Región de Murcia,Región de Murcia
2,Palma de Mallorca,Islas Baleares,Islas Baleares
3,Gijón,Principado de Asturias,Principado de Asturias
4,Oviedo,Principado de Asturias,Principado de Asturias


In [112]:
df_region["municipio"].nunique() == df_region.shape[0]

True

In [120]:
cur.execute("SELECT provincia, id_provincia FROM provincias") #el orden es así para que salga en el mismo orden que en la tabla
provincia_dict = dict(cur.fetchall()) #convierte en un alista de tuplas todo el execute de arriba.
provincia_dict


{'Comunidad de Madrid': 327,
 'Región de Murcia': 328,
 'Islas Baleares': 329,
 'Principado de Asturias': 330,
 'Navarra': 331,
 'Almería': 332,
 'Cantabria': 333,
 'La Rioja': 334,
 'Melilla': 335,
 'Ceuta': 336}

In [131]:
data_to_insert = []

for _, row in df_region.iterrows():
    municipio = row["municipio"]
    id_provincia = provincia_dict.get(row["provincia"])

    data_to_insert.append([municipio, id_provincia])

    #falta meter latitud y longitud

In [128]:
insert_query = """ 
                    INSERT INTO municipios (municipio, id_provincia)
                    values (%s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

### VENTAS

In [132]:
df_ventas.columns

Index(['id_venta', 'fecha_venta', 'fecha_entrega_prevista',
       'fecha_entrega_real', 'id_cliente', 'fecha_nacimiento', 'mail',
       'categoria_producto', 'nombre_producto', 'precio_unitario', 'descuento',
       'cantidad_comprada', 'municipio', 'monto_total', 'nombre', 'apellido'],
      dtype='object')

In [ ]:
cur.close()
conn.close()